In [1]:
import camelot
import numpy as np
import pandas as pd
from WindPy import w

w.start() # 默认命令超时时间为120秒，如需设置超时时间可以加入waitTime参数，例如waitTime=60,即设置命令超时时间为60秒 

w.isconnected() # 判断WindPy是否已经登录成功

tables = camelot.read_pdf('2020-06-23-300045.SZ-华力创通_2020年半年度业绩预告.pdf') #读取pdf

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
# 命令如何写可以用命令生成器来辅助完成
# 定义打印输出函数，用来展示数据使用
def printpy(outdata):
    if outdata.ErrorCode!=0:
        print('error code:'+str(outdata.ErrorCode)+'\n');
        return();
    for i in range(0,len(outdata.Data[0])):
        strTemp=''
        if len(outdata.Times)>1:
            strTemp=str(outdata.Times[i])+' '
        for k in range(0, len(outdata.Fields)):
            strTemp=strTemp+str(outdata.Data[k][i])+' '
        print(strTemp)

In [3]:
def moneyconvert(amount):
    if amount >= 10000 and amount < 100000000:
        val = round(amount/10000,2)
        value = str(val) + "万元"
    elif amount >= 100000000:
        val = round(amount/100000000,2)
        value = str(val) + "亿元"
        
    return value 

In [4]:
tables
#tables.export('2020-07-07-300229.SZ-拓尔思_2020年半年度业绩预告.xlsx', f='excel', compress=False) # 导出excel表格

<TableList n=1>

In [5]:
tables[0] #显示表格的dimension

<Table shape=(3, 3)>

In [6]:
tables[0].parsing_report #表格的精确度

{'accuracy': 100.0, 'whitespace': 22.22, 'order': 1, 'page': 1}

In [7]:
#tables[0].to_excel('2020-06-23-300045.SZ-华力创通_2020年半年度业绩预告.xlsx') # to_json, to_excel, to_html

In [8]:
tables[0].df # get a pandas DataFrame!

,0,1,2
0,项 目,本报告期,上年同期
1,归属于上市公\n司股东的净利\n润,比上年同期增长：50% - 70%,"盈利：2,012.71 万元"
2,,"盈利：3,019.07 万元–3,421.61 万元",


In [9]:
data = tables[0].df.values #写入表格的数值

In [10]:
np.shape(data)

(3, 3)

In [11]:
data_dict = {'obj':data[1,0],'this':data[2,1],'last':data[1,2],'varpct':data[1,1]} #把数据写入dict

data_dict 

{'obj': '归属于上市公\n司股东的净利\n润',
 'this': '盈利：3,019.07 万元–3,421.61 万元',
 'last': '盈利：2,012.71 万元',
 'varpct': '比上年同期增长：50% - 70%'}

In [12]:
# 1 盈利
#预计20H1归母净利xx亿元至xx亿元，中值xx亿元；19H1为盈利xx亿元，同比下降xx%-xx%，中值为下降xx%。
#据此推算，预计20Q2归母净利xx万元至xx亿元，中值xx亿元；19Q2为盈利xx亿元，同比下降xx%-xx%，中值为下降xx%。


In [13]:
# 2 盈利
#预计20H1归母净利xx亿元至xx亿元，中值xx亿元；19H1为盈利xx亿元，同比增长xx%-xx%，中值为增长xx%。
#据此推算，预计20Q2归母净利xx万元至xx亿元，中值xx亿元；19Q2为盈利xx亿元，同比下降xx%-xx%，中值为下降xx%。


In [14]:
#string processing to get the two number and then take mean

a1 = data_dict["this"] #"盈利：3,019.07 万元–3,421.61 万元"
aa1 = a1.replace("–","至")
aaa1 = aa1[3:].split("万元")
aaaa1 = aaa1[0].replace(',','')
# x  = 归母净利下限
x = float(aaaa1)


In [15]:
q = aaa1[1].replace('至','')
qq = q.replace(',','')
# y  = 归母净利上限
y = float(qq)

#计算归母净利的平均值
mean1 = 0.5*(x + y)



In [16]:
b1 = data_dict["last"]
bb1 = b1[3:].split("万元")
# z = 去年的归母净利
z = bb1[0]


In [17]:
bb1dash = data_dict["last"].replace('：','')
bbb1dash = bb1dash.replace(',','')


In [18]:

c1 = data_dict["varpct"]
cc1 = c1[8:].split('-')
ccc1 = cc1[0].replace('%','')
cccc1 = cc1[1].replace('%','')

#计算同比增长率的平均数
mean_pct = 0.5*(float(ccc1) + float(cccc1))


In [19]:
wsddata2 = w.wsd("300045.SZ", "np_belongto_parcomsh", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata2

.ErrorCode=0
.Codes=[300045.SZ]
.Fields=[NP_BELONGTO_PARCOMSH]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[10206910.92,20127056.97,41304664.72,-147549739.89,20730810.13]]

In [20]:
# #转成dataframe
# fm=pd.DataFrame(wsddata2.Data,index=wsddata2.Fields,columns=wsddata2.Times)
# fm=fm.T #转置
# fm.loc[fm.index[0],'NP_BELONGTO_PARCOMSH'] #选取2019年q1数据
# #fm.values[0]
# #fm.index

In [21]:
lastq1 = wsddata2.Data[0][0] #选取2019年q1数据
lasth1 = wsddata2.Data[0][1] #选取2019年h1数据
thisq1 = wsddata2.Data[0][4] #选取2020年q1数据
lastq2 = lasth1 - lastq1     #计算2019年q2数据

thish1_lower = x * 10000
thish1_upper = y * 10000

thisq2_lower = thish1_lower - thisq1     #计算2020年q2下限
thisq2_upper = thish1_upper - thisq1     #计算2020年q2上限
thisq2_mean = 0.5*(thisq2_lower+thisq2_upper)

#计算同比增长/减少
growth_year_lower = (thisq2_lower - lastq2)/abs(lastq2) * 100
growth_year_upper = (thisq2_upper - lastq2)/abs(lastq2) * 100
growth_year_mean = 0.5*(growth_year_lower+growth_year_upper)

In [22]:
line1 = "预计20H1归母净利" + str(x) + "万元" + str(y) + "万元" + '，' + "中值" + str(mean1) + "万元" + '；' + "19H1为" + bbb1dash + '，' +"同比增长"+ ccc1 + '%' + '-' + cccc1 + '%' + '，'+ "中值为增长" + str(mean_pct)+'%'

line2 = "据此推算，预计20Q2归母净利" + moneyconvert(thisq2_lower) + moneyconvert(thisq2_upper)  + '，' + "中值" + moneyconvert(thisq2_mean) + '；' + "19Q2为盈利" + moneyconvert(lastq2) + "，同比增长" + str(round(growth_year_lower,2)) + '%' + '-' + str(round(growth_year_upper,2)) + '%' + '，'+ "中值为增长" + str(round(growth_year_mean,2))+'%'

print(line1)
print(line2)



预计20H1归母净利3019.07万元3421.61万元，中值3220.34万元；19H1为盈利2012.71 万元，同比增长50 %- 70%，中值为增长60.0%
据此推算，预计20Q2归母净利945.99万元1348.53万元，中值1147.26万元；19Q2为盈利992.01万元，同比增长-4.64%-35.94%，中值为增长15.65%
